In [ ]:
import os
os.chdir('..')
import warnings
warnings.filterwarnings('ignore')


Mount the Drive

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')


Mounted at /content/drive


Install Necessary Libraries

Data Loading

In [3]:
from sklearn.model_selection import train_test_split

# Load the CoNLL file
def load_conll_data(filepath):
    sentences = []
    sentence = []
    with open(filepath, 'r', encoding='utf-8') as file:
        for line in file:
            if line.strip() == "":
                if sentence:
                    sentences.append(sentence)
                    sentence = []
            else:
                token, label = line.strip().split()
                sentence.append((token, label))
    if sentence:
        sentences.append(sentence)
    return sentences

# filepath = 'drive/MyDrive/Colab Notebooks/labeled_data.conll'
filepath = 'labeled_data.conll'
sentences = load_conll_data(filepath)

# Split into training and testing data
train_data, test_data = train_test_split(sentences, test_size=0.2, random_state=42)


In [4]:
print(sentences)


[[('ለመላው', 'O'), ('የክርስትና', 'O'), ('እምነት', 'O'), ('ተከታዮች', 'O'), ('በሙሉ', 'O'), ('እንኳን', 'O'), ('ለብርሃነ', 'O'), ('መስቀሉ', 'O'), ('በሰላም', 'O'), ('አደረሳችሁ', 'O'), ('አደረሰን', 'O'), ('በዓሉ', 'O'), ('የሰላም', 'O'), ('የጤና', 'O'), ('የፍቅር', 'O'), ('የደስታ', 'O'), ('እና', 'O'), ('የመተሳሰብ', 'O'), ('በዓል', 'O'), ('ይሁንልን', 'O'), ('መልካም', 'O'), ('የመስቀል', 'O'), ('በዓል', 'O')], [('የሞተ', 'B-Product'), ('ቆዳን', 'I-Product'), ('እንዲሁም', 'O'), ('ቆሻሻን', 'I-Product'), ('ለማፅዳት', 'I-Product'), ('ተመራጭ', 'O'), ('ዋጋ', 'B-PRICE'), ('200', 'I-PRICE'), ('ብር', 'B-PRICE'), ('ውስን', 'O'), ('ፍሬ', 'O'), ('ነው', 'O'), ('ያለው', 'O'), ('አድራሻ', 'B-LOC'), ('ቁ', 'I-LOC'), ('1', 'I-LOC'), ('መገናኛ', 'I-LOC'), ('መሰረት', 'I-LOC'), ('ደፋር', 'I-LOC'), ('ሞል', 'I-LOC'), ('ሁለተኛ', 'I-LOC'), ('ፎቅ', 'I-LOC'), ('ቢሮ', 'I-LOC'), ('ቁ', 'I-LOC'), ('05', 'I-LOC'), ('06', 'I-LOC'), ('ቁ', 'I-LOC'), ('2', 'I-LOC'), ('ፒያሳ', 'I-LOC'), ('ጊዮርጊስ', 'I-LOC'), ('አደባባይ', 'I-LOC'), ('ራመት_ታቦር_ኦዳ_ህንፃ', 'I-LOC'), ('1ኛ', 'I-LOC'), ('ፎቅ', 'I-LOC'), ('ሱቅ', 'I-LOC'), ('ቁ', 'I-LOC'), 

In [5]:
print(train_data)


In [6]:
print(test_data)


[[('በረፍት', 'O'), ('ቀንዎ', 'O'), ('ሱቅ', 'B-LOC'), ('ላይ', 'O'), ('መስተናገድ', 'O'), ('ለምትፈልጉ', 'O'), ('ውድ', 'O'), ('ደንበኞቻችን', 'O'), ('ልዮ', 'B-Product'), ('የዋዜማ', 'O'), ('ገበያ', 'O'), ('ከቅናሽ', 'O'), ('ጋር', 'O'), ('አዘጋጅተን', 'O'), ('እንጠብቅዎታለን', 'O'), ('ነገ', 'O'), ('ከጠዋቱ', 'O'), ('4', 'O'), ('30', 'O'), ('_', 'O'), ('ቀኑ', 'O'), ('11', 'O'), ('00', 'O'), ('ድረስ', 'O'), ('ሱቃችን', 'O'), ('ክፍት', 'O'), ('ሁኖ', 'O'), ('ይጠብቅዎታል', 'O'), ('ሱቅ', 'I-LOC'), ('መተው', 'O'), ('መግዛት', 'O'), ('ላልቻላችሁ', 'O'), ('በሞተረኞች', 'O'), ('ያሉበት', 'O'), ('እናደርሳለን', 'O'), ('ዘወትር', 'O'), ('ሰኞ', 'O'), ('_ቅዳሜ', 'O'), ('ከጠዋቱ', 'O'), ('2', 'I-LOC'), ('30', 'O'), ('እስከ', 'O'), ('ምሽቱ', 'O'), ('2', 'I-LOC'), ('00', 'O'), ('ድረስ', 'O'), ('ክፍት', 'O'), ('መሆኑን', 'O'), ('እንገልፃለን', 'O'), ('አድራሻ', 'I-LOC'), ('ቁ', 'I-LOC'), ('1', 'I-LOC'), ('መገናኛ', 'I-LOC'), ('መሰረት', 'I-LOC'), ('ደፋር', 'I-LOC'), ('ሞል', 'I-LOC'), ('ሁለተኛ', 'I-LOC'), ('ፎቅ', 'I-LOC'), ('ቢሮ', 'I-LOC'), ('ቁ', 'I-LOC'), ('05', 'I-LOC'), ('06', 'I-LOC'), ('ቁ', 'I-LOC'), ('2', 'I-LOC'), ('ፒያ

In [7]:
model_names = {
    "xlm-roberta": "xlm-roberta-base",
    "distilbert": "distilbert-base-multilingual-cased",
    "mbert": "bert-base-multilingual-cased"
}


# Pre processing

In [8]:
from datasets import Dataset, load_dataset,DatasetDict

label_list = ['O', 'B-Product', 'I-Product', 'B-PRICE', 'I-PRICE', 'B-LOC', 'I-LOC']
label2id = {label: i for i, label in enumerate(label_list)}
id2label = {i: label for i, label in enumerate(label_list)}

# Function to map labels to integers
def encode_labels(labels):
    return [label2id[label] for label in labels]
def convert_to_dataset(data):
    tokens = []
    labels = []

    for sentence in data:
        sentence_tokens = [pair[0] for pair in sentence]
        sentence_labels = encode_labels([pair[1] for pair in sentence])
        tokens.append(sentence_tokens)
        labels.append(sentence_labels)

    return Dataset.from_dict({"tokens": tokens, "labels": labels})

train_dataset = convert_to_dataset(train_data)
test_dataset = convert_to_dataset(test_data)

# # Create DatasetDict for train and test
datasets = DatasetDict({
    'train': train_dataset,
    'test': test_dataset
})




In [9]:
from transformers import AutoTokenizer
from datasets import Dataset, DatasetDict

model = {}
tokenizer = {}
for key, model_name in model_names.items():
  model[key] = model_name
  tokenizer[key] = AutoTokenizer.from_pretrained(model_name)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

In [10]:
print(model)


{'xlm-roberta': 'xlm-roberta-base', 'distilbert': 'distilbert-base-multilingual-cased', 'mbert': 'bert-base-multilingual-cased'}


In [11]:
print(tokenizer['xlm-roberta'])


XLMRobertaTokenizerFast(name_or_path='xlm-roberta-base', vocab_size=250002, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	250001: AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False, special=True),
}


In [12]:
def tokenize_and_align_labels(datasets,tokenizer):
    tokenized_inputs = tokenizer(
        datasets["tokens"],
        truncation=True,
        padding="max_length",
        max_length=128,
        is_split_into_words=True
    )

    labels = []
    for i, label in enumerate(datasets["labels"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        aligned_labels = []
        previous_word_idx = None
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 for them.
            if word_idx is None:
                aligned_labels.append(-100)
            # Only label the first token of a given word.
            elif word_idx != previous_word_idx:
                aligned_labels.append(label[word_idx])
            # Set the label to -100 for the other subtokens of the word.
            else:
                aligned_labels.append(-100)
            previous_word_idx = word_idx

        labels.append(aligned_labels)

    tokenized_inputs["labels"] = labels
    # labelss=labels
    return tokenized_inputs
# Tokenize the datasets

tokenized_datasets = {}
for key, model_name in model_names.items():
  tokenized_datasets[key] = datasets.map(tokenize_and_align_labels, batched=True,fn_kwargs={'tokenizer': tokenizer[key]})



Map:   0%|          | 0/130 [00:00<?, ? examples/s]

Map:   0%|          | 0/33 [00:00<?, ? examples/s]

Map:   0%|          | 0/130 [00:00<?, ? examples/s]

Map:   0%|          | 0/33 [00:00<?, ? examples/s]

Map:   0%|          | 0/130 [00:00<?, ? examples/s]

Map:   0%|          | 0/33 [00:00<?, ? examples/s]

In [13]:
print(tokenized_datasets)


{'xlm-roberta': DatasetDict({
    train: Dataset({
        features: ['tokens', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 130
    })
    test: Dataset({
        features: ['tokens', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 33
    })
}), 'distilbert': DatasetDict({
    train: Dataset({
        features: ['tokens', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 130
    })
    test: Dataset({
        features: ['tokens', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 33
    })
}), 'mbert': DatasetDict({
    train: Dataset({
        features: ['tokens', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 130
    })
    test: Dataset({
        features: ['tokens', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 33
    })
})}


In [14]:
print(tokenized_datasets['xlm-roberta']['train'])


Dataset({
    features: ['tokens', 'labels', 'input_ids', 'attention_mask'],
    num_rows: 130
})


In [15]:
print("Sample tokenized input with labels:", tokenized_datasets['xlm-roberta']['train'][0])
print("Length of tokens:", len(tokenized_datasets['xlm-roberta']['train'][0]["input_ids"]))
print("Length of labels:", len(tokenized_datasets['xlm-roberta']['train'][0]["labels"]))


Sample tokenized input with labels: {'tokens': ['በረፍት', 'ቀንዎ', 'ሱቅ', 'ላይ', 'መስተናገድ', 'ለምትፈልጉ', 'ውድ', 'ደንበኞቻችን', 'ልዮ', 'የዋዜማ', 'ገበያ', 'ከቅናሽ', 'ጋር', 'አዘጋጅተን', 'እንጠብቅዎታለን', 'ነገ', 'ከጠዋቱ', '4', '30', '_', 'ቀኑ', '11', '00', 'ድረስ', 'ሱቃችን', 'ክፍት', 'ሁኖ', 'ይጠብቅዎታል', 'ሱቅ', 'መተው', 'መግዛት', 'ላልቻላችሁ', 'በሞተረኞች', 'ያሉበት', 'እናደርሳለን', 'ዘወትር', 'ሰኞ', '_ቅዳሜ', 'ከጠዋቱ', '2', '30', 'እስከ', 'ምሽቱ', '2', '00', 'ድረስ', 'ክፍት', 'መሆኑን', 'እንገልፃለን', 'አድራሻ', 'ቁ', '1', 'መገናኛ', 'መሰረት', 'ደፋር', 'ሞል', 'ሁለተኛ', 'ፎቅ', 'ቢሮ', 'ቁ', '05', '06', 'ቁ', '2', 'ፒያሳ', 'ጊዮርጊስ', 'አደባባይ', 'ራመት_ታቦር_ኦዳ_ህንፃ', '1ኛ', 'ፎቅ', 'ሱቅ', 'ቁ', '1', '-107', '0902660722', '0928460606', 'ፒያሳ', 'ቅርንጫፍ', '0941337070', 'በ', 'ለማዘዝ', 'ይጠቀሙ', 'ለተጨማሪ', 'ማብራሪያ', 'የቴሌግራም', 'ገፃችን'], 'labels': [-100, 0, -100, -100, 0, -100, 5, -100, -100, 0, 0, -100, -100, 0, -100, -100, -100, -100, 0, 0, -100, -100, -100, -100, 1, -100, 0, -100, -100, 0, -100, 0, -100, -100, 0, 0, -100, 0, -100, -100, -100, -100, 0, 0, -100, -100, -100, 0, 0, 0, 0, -100, 0, 0, 0, 0, -100, -100, -100, 0, -1

In [16]:
from transformers import XLMRobertaForTokenClassification, DistilBertForTokenClassification, BertForTokenClassification
from transformers import Trainer, TrainingArguments
models = {}
models['xlm-roberta'] = XLMRobertaForTokenClassification.from_pretrained("xlm-roberta-base", num_labels=len(label_list))
models["distilbert"] = DistilBertForTokenClassification.from_pretrained("distilbert-base-multilingual-cased", num_labels=len(label_list))
models["mbert"] = BertForTokenClassification.from_pretrained("bert-base-multilingual-cased", num_labels=len(label_list))


model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model.safetensors:   0%|          | 0.00/542M [00:00<?, ?B/s]

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
print(models['distilbert'])


DistilBertForTokenClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
   

In [19]:
from transformers import Trainer, TrainingArguments
from seqeval.metrics import classification_report


def fine_tune_model(model, model_name):
  training_args = TrainingArguments(
      output_dir=f"./results/{model_name}",
      evaluation_strategy="epoch",
      logging_strategy="steps",
      logging_steps=3,
      learning_rate=2e-5,
      per_device_train_batch_size=8,
      per_device_eval_batch_size=8,
      num_train_epochs=20,
      weight_decay=0.01,
      logging_dir='./logs',
  )
  # Define a Trainer
  trainer = Trainer(
      model=model,
      args=training_args,
      train_dataset=tokenized_datasets[model_name]['train'],
      eval_dataset=tokenized_datasets[model_name]['test'],
      tokenizer=tokenizer[model_name],
  )

  # Train the model
  trainer.train()
  return trainer



In [20]:
roberta_trainer = fine_tune_model(models['xlm-roberta'], 'xlm-roberta')


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.524800,0.394173
2,0.268900,0.231178
3,0.139800,0.168328
4,0.087500,0.135329
5,0.083600,0.116611
6,0.134600,0.085153
7,0.038700,0.059850
8,0.042600,0.044587
9,0.010200,0.048364
10,0.025600,0.037924


# Model Training

In [21]:
distilbert_trainer = fine_tune_model(models['distilbert'], 'distilbert')


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.816800,0.617479
2,0.414900,0.360369
3,0.275500,0.247858
4,0.209100,0.215328
5,0.162000,0.207124
6,0.232000,0.191072
7,0.117400,0.177473
8,0.120400,0.173181
9,0.076500,0.167601
10,0.153100,0.163553


In [22]:
mbert_trainer = fine_tune_model(models['mbert'], 'mbert')


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.607400,0.454348
2,0.296900,0.268956
3,0.204000,0.218104
4,0.142800,0.208189
5,0.145300,0.197126
6,0.221000,0.195400
7,0.115500,0.182804
8,0.103100,0.176706
9,0.058800,0.175015
10,0.117900,0.167165


# Evaluation

In [23]:
import numpy as np
from seqeval.metrics import classification_report

pred_data = {'xlm-roberta':roberta_trainer,'distilbert':distilbert_trainer,'mbert':mbert_trainer}
# Make predictions on the test set
for model_name,trainer in pred_data.items():
  predictions, labels, _ = trainer.predict(tokenized_datasets[model_name]["test"])
  predictions = np.argmax(predictions, axis=2)

  # Convert predictions and labels to original label format
  true_labels = [[label_list[l] for l in label if l != -100] for label in labels]
  predicted_labels = [[label_list[p] for (p, l) in zip(pred, label) if l != -100] for pred, label in zip(predictions, labels)]

  # Evaluate using seqeval
  report = classification_report(true_labels, predicted_labels)
  print(f'{model_name} evaluation \n{report}\n\n')


xlm-roberta evaluation 
              precision    recall  f1-score   support

         LOC       1.00      1.00      1.00        68
       PRICE       1.00      1.00      1.00        59
     Product       0.97      0.94      0.96        36

   micro avg       0.99      0.99      0.99       163
   macro avg       0.99      0.98      0.99       163
weighted avg       0.99      0.99      0.99       163





distilbert evaluation 
              precision    recall  f1-score   support

         LOC       0.93      0.89      0.91        76
       PRICE       0.97      0.92      0.94        63
     Product       0.00      0.00      0.00        36

   micro avg       0.93      0.72      0.81       175
   macro avg       0.63      0.61      0.62       175
weighted avg       0.75      0.72      0.74       175





mbert evaluation 
              precision    recall  f1-score   support

         LOC       0.92      0.92      0.92        76
       PRICE       0.98      0.92      0.95        63
     Product       0.17      0.06      0.08        36

   micro avg       0.88      0.74      0.81       175
   macro avg       0.69      0.63      0.65       175
weighted avg       0.79      0.74      0.76       175





# Save The Model

In [24]:
best_model_name = 'xlm-roberta'


In [25]:
models[best_model_name].save_pretrained("drive/MyDrive/Colab Notebooks/finetuned-model")
tokenizer[best_model_name].save_pretrained("drive/MyDrive/Colab Notebooks/finetuned-model")


('drive/MyDrive/Colab Notebooks/finetuned-model/tokenizer_config.json',
 'drive/MyDrive/Colab Notebooks/finetuned-model/special_tokens_map.json',
 'drive/MyDrive/Colab Notebooks/finetuned-model/sentencepiece.bpe.model',
 'drive/MyDrive/Colab Notebooks/finetuned-model/added_tokens.json',
 'drive/MyDrive/Colab Notebooks/finetuned-model/tokenizer.json')

# Inference on New Data

In [26]:
from transformers import AutoModelForTokenClassification, AutoTokenizer, XLMRobertaForTokenClassification
import torch
import numpy as np

# Load the saved model and tokenizer
model = XLMRobertaForTokenClassification.from_pretrained("drive/MyDrive/Colab Notebooks/finetuned-model")
tokenizer = AutoTokenizer.from_pretrained("drive/MyDrive/Colab Notebooks/finetuned-model")

# New sentences to test
new_sentences = ["""ዋጋ 200 ብር አድራሻ መገናኛ መሰረት ደፋር ሞል ውስን ፍሬ ነው ያለው ልጆች እየተዝናኑ የሚማሩበት
ለፅሁፍ እና ለስዕል መለማመጃ የሚሆን
የወረቀት እና እርሳስ ወጪን የሚቆጥብ
ብናኝ ወይም አይን የሚወጋ ብርሀን ስለሌለው በምቾት እና በደህንነት ለረጅም ሰአት የሚጠቀሙበት"""]

# Tokenize the new sentences into words
tokenized_inputs = tokenizer(
    new_sentences,
    padding=True,
    truncation=True,
    return_tensors="pt",
    is_split_into_words=False
)

# Convert ids back to tokens for debugging
tokens = tokenizer.convert_ids_to_tokens(tokenized_inputs["input_ids"][0])

model.eval()

# Disable gradient calculations
with torch.no_grad():
    outputs = model(**tokenized_inputs)

# Get the predictions
logits = outputs.logits
predictions = torch.argmax(logits, dim=2)

# Define the label list (ensure it matches the one used during training)
label_list = ['O', 'B-Product', 'I-Product', 'B-PRICE', 'I-PRICE', 'B-LOC', 'I-LOC']

# Mapping predictions back to words
predicted_labels = []
for i, pred in enumerate(predictions):
    word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to original words
    current_word_labels = []

    prev_word_id = None
    word_label = None
    for idx, word_id in enumerate(word_ids):
        if word_id is None:
            continue
        # If this is a new word
        if word_id != prev_word_id:
            # Append label of the previous word if it exists
            if word_label is not None:
                current_word_labels.append(label_list[word_label])
            # Get the label for the new word (use the label for the first subword)
            word_label = pred[idx].item()
        prev_word_id = word_id

    # Append the last word's label
    if word_label is not None:
        current_word_labels.append(label_list[word_label])

    predicted_labels.append(current_word_labels)

# Print the predicted labels for each word in the sentence
for sentence, labels in zip(new_sentences, predicted_labels):
    # Split the sentence into words
    words = sentence.split()

    # Print labels for each word
    print(f"Sentence: {sentence}")
    print("Word-Level Labels:")
    for word, label in zip(words, labels):
        print(f"{word}: {label}")


Sentence: ዋጋ 200 ብር አድራሻ መገናኛ መሰረት ደፋር ሞል ውስን ፍሬ ነው ያለው ልጆች እየተዝናኑ የሚማሩበት
ለፅሁፍ እና ለስዕል መለማመጃ የሚሆን
የወረቀት እና እርሳስ ወጪን የሚቆጥብ
ብናኝ ወይም አይን የሚወጋ ብርሀን ስለሌለው በምቾት እና በደህንነት ለረጅም ሰአት የሚጠቀሙበት
Word-Level Labels:
ዋጋ: B-PRICE
200: I-PRICE
ብር: B-PRICE
አድራሻ: B-LOC
መገናኛ: I-LOC
መሰረት: I-LOC
ደፋር: I-LOC
ሞል: O
ውስን: O
ፍሬ: O
ነው: O
ያለው: O
ልጆች: O
እየተዝናኑ: O
የሚማሩበት: I-Product
ለፅሁፍ: O
እና: O
ለስዕል: O
መለማመጃ: O
የሚሆን: O
የወረቀት: O
እና: O
እርሳስ: O
ወጪን: O
የሚቆጥብ: O
ብናኝ: O
ወይም: O
አይን: O
የሚወጋ: O
ብርሀን: O
ስለሌለው: O
በምቾት: O
እና: O
በደህንነት: O
ለረጅም: O
ሰአት: O
የሚጠቀሙበት: O


# Model Interpretability

In [32]:
# !pip install shap


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 540.1/540.1 kB 16.6 MB/s eta 0:00:00


In [33]:
# !pip install lime


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 9.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283834 sha256=8a8a65dc7f24499f2fa224102db5bf2ed4c278fb02789966652c6f0cdce8c7cd
  Stored in directory: /root/.cache/pip/wheels/fd/a2/af/9ac0a1a85a27f314a06b39e1f492bee1547d52549a4606ed89
Successfully built lime


In [34]:
# Import necessary libraries
import numpy as np
import pandas as pd
import shap
import lime
import lime.lime_text
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification
from sklearn.metrics import classification_report

# Load your fine-tuned model and tokenizer
model_name = "drive/MyDrive/Colab Notebooks/finetuned-model"
tokenizer = AutoTokenizer.from_pretrained("drive/MyDrive/Colab Notebooks/finetuned-model")
model = AutoModelForTokenClassification.from_pretrained(model_name)

